In [8]:
import numpy as np

data = np.load(r"data\jkjkjk95\平静\IR_camera\1755928035.7015848f30c894.npz")
frames = data['frames']
frame_lens = data['frame_lens']

In [9]:
import cv2
for i,frame in enumerate(frames):
    frame_data = frame[:frame_lens[i]]
    # cv2解码
    img = cv2.imdecode(frame_data, cv2.IMREAD_GRAYSCALE)
    cv2.imshow('img',img)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
cv2.destroyAllWindows()
